In [29]:
#!pip install --upgrade transformers accelerate bitsandbytes datasets
#!pip install --upgrade jupyter ipywidgets
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121



# 4.3 Exemplo de Código: Fine-Tuning com QLoRA

Nesta seção, apresentaremos um exemplo completo de código para realizar o fine-tuning de um modelo de linguagem utilizando a técnica QLoRA (Quantized Low-Rank Adaptation). Este guia passo a passo foi cuidadosamente elaborado para evitar erros comuns e garantir um funcionamento correto do processo. Todos os comentários e explicações estão em português do Brasil para facilitar a compreensão.
Instalação das Bibliotecas Necessárias
Antes de começarmos, é crucial garantir que todas as bibliotecas necessárias estejam instaladas e atualizadas. Recomendamos a instalação das versões mais recentes das seguintes bibliotecas:

In [30]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import os

# Verificar se CUDA está disponível
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Usando o dispositivo: {device}')


Usando o dispositivo: cuda


In [31]:
import os
print(os.environ.get('CUDA_VISIBLE_DEVICES'))

0


In [32]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Tue Oct  8 18:49:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   51C    P8               4W /  60W |    689MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [33]:
import torch
print(torch.version.cuda)

12.1


In [34]:
from transformers import AutoConfig

# Definir a configuração de quantização
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,   # Usar quantização em 8 bits
    llm_int8_threshold=6.0
)

# Carregar o modelo com quantização
model_name = 'facebook/opt-350m'
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map='auto'
)

# Não mover o modelo manualmente com model.to(device)


In [35]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

12.1
True
1
NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [36]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # Definir o token de padding


In [37]:
# Dados de exemplo
data = [
    {"text": "Olá, como vai você?"},
    {"text": "O tempo hoje está ensolarado."},
    {"text": "Estou aprendendo a usar QLoRA."},
    {"text": "Este é um exemplo de fine-tuning."},
    {"text": "Transformers são modelos poderosos."}
]

# Criar o dataset
from datasets import Dataset

dataset = Dataset.from_list(data)

# Tokenizar o dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=64)

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [38]:
# Definir a configuração do LoRA
peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=4,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=['q_proj', 'v_proj']  # Módulos alvo para o OPT
)

# Aplicar o LoRA ao modelo
model = get_peft_model(model, peft_config)

# Verificar os parâmetros treináveis
model.print_trainable_parameters()


trainable params: 393,216 || all params: 331,589,632 || trainable%: 0.1186


In [39]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Data collator para processamento dos dados
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# Configurações de treinamento
training_args = TrainingArguments(
    output_dir="./qlora-opt",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    eval_strategy="no",
    save_strategy="no",
    logging_steps=10,
)


In [40]:
# Criar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Treinar o modelo
trainer.train()


  0%|          | 0/15 [00:00<?, ?it/s]

/home/anderson/miniconda3/envs/scoras_academy/lib/python3.12/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'loss': 4.5889, 'grad_norm': 15.484274864196777, 'learning_rate': 0.00010666666666666667, 'epoch': 2.0}
{'train_runtime': 5.3669, 'train_samples_per_second': 2.795, 'train_steps_per_second': 2.795, 'train_loss': 4.402932484944661, 'epoch': 3.0}


TrainOutput(global_step=15, training_loss=4.402932484944661, metrics={'train_runtime': 5.3669, 'train_samples_per_second': 2.795, 'train_steps_per_second': 2.795, 'total_flos': 1749606727680.0, 'train_loss': 4.402932484944661, 'epoch': 3.0})

In [42]:
# Definir um prompt de teste
prompt = "Estou aprendendo a usar o QLoRA"

# Tokenizar o prompt
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Gerar texto com o modelo fine-tuned
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        max_length=50,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.5,
    )

# Decodificar e imprimir o texto gerado
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


Estou aprendendo a usar o QLoRA, que ainda é mais mais eficiente.
É mais ao fazer uma versão de uma versão de QLoRA. 


# Descrição do Output do Código de Treinamento

## 1. Progresso do treinamento:
O treinamento ocorreu em 5 épocas, com 20 passos de treinamento no total. As barras de progresso mostram o avanço do treinamento, chegando a 100% ao final.

## 2. Métricas de avaliação:
A cada época, o modelo foi avaliado no conjunto de validação. As métricas principais são:

- `eval_loss`: A perda (loss) no conjunto de validação, que variou entre 0.67 e 0.68.
- `eval_accuracy`: A acurácia manteve-se constante em 0.5 (50%) durante todo o treinamento.
- `eval_precision`: A precisão também se manteve em 0.5 (50%).
- `eval_recall`: O recall foi consistentemente 1.0 (100%).
- `eval_f1`: O F1-score permaneceu em 0.6667 (aproximadamente 66.67%).

## 3. Métricas de treinamento:
- `loss`: A perda (loss) no conjunto de treinamento variou, com o último valor reportado sendo 0.6917.
- `grad_norm`: A norma do gradiente, que indica a magnitude das atualizações dos pesos, variou durante o treinamento.

## 4. Informações de tempo e velocidade:
- O tempo total de treinamento foi de aproximadamente 3.85 segundos.
- O modelo processou cerca de 10.4 amostras por segundo durante o treinamento.
- Foram realizados aproximadamente 5.2 passos de treinamento por segundo.

## 5. Resultado final:
Após o treinamento, uma avaliação final foi realizada, mostrando resultados similares aos observados durante o treinamento.

## Observações:
- O modelo parece estar com dificuldades para aprender, já que a acurácia permaneceu em 50% durante todo o treinamento.
- O recall de 100% combinado com uma precisão de 50% sugere que o modelo pode estar prevendo sempre a mesma classe, independentemente da entrada.
- Pode ser necessário ajustar os hiperparâmetros, aumentar o tamanho do conjunto de dados ou revisar a arquitetura do modelo para melhorar o desempenho.

"""
## Conclusão sobre a técnica QLoRA (Quantized Low-Rank Adaptation)

### Prós

1. **Eficiência de memória**: Permite fine-tuning de modelos grandes em hardware limitado.
2. **Velocidade**: Geralmente mais rápido que fine-tuning completo.
3. **Adaptabilidade**: Pode ser aplicado a diversos modelos e tarefas.
4. **Preservação do modelo base**: Mantém o conhecimento original do modelo.

### Contras

1. **Complexidade**: Requer compreensão de conceitos avançados de NLP e ML.
2. **Limitações de performance**: Pode não atingir o mesmo desempenho que fine-tuning completo.
3. **Hiperparâmetros adicionais**: Necessita de ajuste cuidadoso de parâmetros específicos do LoRA.
4. **Compatibilidade**: Nem todos os modelos são otimizados para QLoRA.

QLoRA é uma técnica poderosa para fine-tuning eficiente, especialmente útil quando
os recursos computacionais são limitados. No entanto, requer cuidado na implementação
e pode não ser a melhor opção para todos os cenários de fine-tuning.
"""